# Natural Language Processing with Attention Models

Notas sobre o curso Natural Language Processing with Attention Models da DeeplearninigAI. O notebook é composto majoritariamente de material original, salvo as figuras, que foram criadas pela **Deep Learning AI** e disponibilizadas em seu curso.

# Week 1 - Neural Machine Translation

## Seq2seq

Os modelos Seq2Seq (Sequence-to-Sequence) são amplamente utilizados em tarefas de tradução automática (machine translation) e outras tarefas de NLP que envolvem a conversão de uma sequência de entrada em uma sequência de saída. Eles foram inicialmente propostos para tarefas como tradução automática, onde uma sequência de palavras em uma língua (entrada) é convertida em uma sequência de palavras em outra língua (saída). A arquitetura básica é composta por dois componentes principais:

- **Encoder**: Transforma a sequência de entrada em um vetor de contexto.
- **Decoder**: Utiliza o vetor de contexto para gerar a sequência de saída.

<img src="./imgs/seq2seq.png">

### Arquitetura Seq2Seq

#### Encoder

O encoder é tipicamente uma rede recorrente (RNN, LSTM, GRU) que lê a sequência de entrada token por token e gera um vetor de contexto que representa a informação da sequência inteira.

- **Inputs**: Sequência de entrada $X = (x_1, x_2, ..., x_n)$.
- **Outputs**: Vetor de contexto $C$ e estado oculto $h_t$ em cada passo de tempo.

<img src="./imgs/seq2seq_encoder.png">

#### Decoder

O decoder é outra rede recorrente que gera a sequência de saída um token por vez, **usando o vetor de contexto do encoder** e o **estado oculto anterior do decoder**. A sequencia de entrada no decoder inicia com token $\text{<SOS>}$ (Start of Sequence)

- **Inputs**: O vetor de contexto do encoder, o estado oculto anterior e o token de entrada anterior.
- **Outputs**: Token de saída e novo estado oculto.

<img src="./imgs/seq2seq_decoder.png">

#### Attention

O mecanismo de atenção foi introduzido para melhorar a performance dos modelos Seq2Seq, permitindo que o modelo foque em diferentes partes da entrada enquanto gera cada token da saída.

- **Atenção Básica**: Calcula um conjunto de pesos de atenção que indicam a importância de cada token da sequência de entrada para a geração do próximo token da saída.
- **Context Vector**: Um vetor de contexto ponderado é calculado como uma combinação linear dos estados ocultos do encoder, ponderado pelos pesos de atenção.

Matematicamente, a atenção pode ser descrita da seguinte maneira:
$$ \alpha_{ij} = \frac{\exp(e_{ij})}{\sum_{k=1}^{T_x} \exp(e_{ik})} $$
onde $e_{ij} = a(s_{i-1}, h_j)$ é a função de alinhamento que pode ser um simples perceptron.

<img src="./imgs/seq2seq_attention.png">

### Aplicação em Machine Translation

Na tradução automática, o processo funciona da seguinte maneira:

1. **Encoding**: O encoder lê a sequência de entrada (frase na língua de origem) e gera um vetor de contexto.
2. **Attention**: Durante a decodificação, o mecanismo de atenção calcula os pesos de atenção que focam nas partes relevantes da sequência de entrada.
3. **Decoding**: O decoder gera a sequência de saída (frase na língua de destino), token por token, utilizando o vetor de contexto ponderado pela atenção.

### Exemplo Simplificado

1. **Entrada**: "I love NLP"
2. **Encoding**: O encoder gera vetores de estado oculto para cada token e um vetor de contexto geral.
3. **Attention**: Para cada token gerado na saída, calcula-se a importância de cada token da entrada.
4. **Decoding**: Gera a saída "Yo amo PLN" utilizando os vetores de contexto e atenção.

### Benefícios do Mecanismo de Atenção

- **Melhora a Tradução**: Permite que o modelo foque nas partes mais relevantes da entrada.
- **Mitiga Problemas de Longas Sequências**: Ajuda a manter a informação relevante mesmo em sequências longas.

Os modelos Seq2Seq com atenção são fundamentais para muitas tarefas de NLP, especialmente a tradução automática. A arquitetura básica de encoder-decoder, aprimorada com o mecanismo de atenção, permite que esses modelos lidem eficazmente com a complexidade das sequências de linguagem natural.

## Seq2seq Model with Attention

## Background on seq2seq

## Queries, Keys, Values, and Attention

## Setup for Machine Translation

## Teacher Forcing

## NMT Model with Attention

## BLEU Score

## ROUGE-N Score

## Sampling and Decoding

## Beam Search

## Minimum Bayes Risk

## Ungraded Lab: Stack Semantics

## Neural Machine Translation

## NMT with Attention

# Week 2

# Week 3

# Week 4

# Referência
- Natural Language Processing with Attention Models, disponível em https://www.coursera.org/learn/probabilistic-models-in-nlp

# Licença
- CC BY-SA 2.0 LEGAL CODE. Attribution-ShareAlike 2.0 Generic
- Para detalhes sobre a licença, verifique https://creativecommons.org/licenses/by-sa/2.0/legalcode